# Lab - Collaborative Recommender System

In this lab you will implement the collaborative ﬁltering learning algorithm and apply it to a dataset of movie ratings. 

MovieLens 100k Dataset from GroupLens Research (https://grouplens.org/datasets/movielens/). 

This dataset consists of ratings on a scale of 1 to 5. The dataset has $n_u$ = 943 users, and
$n_m$ = 1682 movies. 

In [2]:
#Load relevant libraries

import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import pandas as pd

  ## Load the data
Load the matrices $Y$ and $R$ from matlab file *ex8_movies.mat*. 
 
$Y$ (*num_movies × num_users*)  stores the ratings $y^{(i,j)}$
(from 1 to 5). 

$R$ is an binary-valued indicator matrix, where $R(i, j)$ = 1 if user $j$ gave a rating to movie $i$, and $R(i, j)$ = 0 otherwise. 

The objective of collaborative ﬁltering is to predict movie ratings for the movies
that users have not yet rated, that is, the entries with $R(i, j) = 0$. This will allow us to recommend the movies with the highest predicted ratings to the user.

In [3]:
# Load Y and R

mat = loadmat("ex8_movies.mat")
mat

Y = mat["Y"]    # 1682 X 943 matrix, containing ratings (1-5) of 1682 movies on 943 user

R = mat["R"]   # 1682 X 943 matrix, R(i,j) = 1 if and only if user j give rating to movie i

# Confirm the shapes of Y and R

print(Y.shape)
print(R.shape)

(1682, 943)
(1682, 943)


In [4]:
# How many users gave a score for the first movie ? ANSWER = 452
print(sum(R[0][:] != 0))

# how many users gave a score for the second movie ? ANSWER= 131
print(sum(R[1][:] != 0))

# How many users gave a score for the last movie ? ANSWER = 1 
print(sum(R[-1][:] != 0))


452
131
1


In [5]:
#Compute the average movie rating for the ﬁrst movie (Toy Story). ANSWER: 3.88
print(sum(Y[0][:])/sum(Y[0][:] != 0))

3.8783185840707963


### Collaborative Filtering Learning Algorithm

The collaborative ﬁltering algorithm for movie recommendations considers that the movies have features, for example x1=romance, x2=action, x3=comedy, etc. In general the movies have $n$ different features, represented as a set of n-dimensional feature vectors X= $[ x^{(1)},... x^{(n_m)}] $. 

The model that will predict what is the rating $y(i,j)$ for movie $i$ given by user $j$   is formulated as a Linear Regression between the movie features and the user parameters Theta= $[ θ^{(1)}, ...,θ^{(n_u)}] $, that is 

$ y(i,j)=(θ^{(j)})^{T}x^{(i)}$ =>  **Linear Regression model for computing the scores for movie $i$ given by user $j$**

Given a dataset of ratings produced
by **some users on some movies**, the model will try to learn simultaneously both vectors  

 X=$ [x^{(1)},... x^{(n_m)}]$ and Theta=$[θ^{(1)}, ...,θ^{(n_u)}]$
 
 that produce the best ﬁt (minimize the
error between the predictions $y(i,j)$ and the real scores given by the users).


**The collaborative filtering cost function with regularization terms is given by**

$J(x^{(1)},...,x^{(n_m)},\theta^{(1)},...,\theta^{(n_u)}) = \frac{1}{2} \sum_{((i,j): r(i,j)=1)}((\theta^{(j)})^Tx^{(i)} - y^{(i,j)})^2 + (\frac{\lambda}{2} \sum^{n_u}_{j=1}\sum^n_{k=1} (\theta^{(j)}_k)^2) + (\frac{\lambda}{2} \sum^{n_m}_{i=1}\sum^n_{k=1} (x^{(i)}_k)^2)$

**The collaborative filtering gradients of the cost function $J$ (with regularization) are given by**

$\frac{\partial J}{\partial x^{(i)}_k} = \sum ((\theta^{(j)})^Tx^{(i)} - y^{(i,j)})\theta_k^{(j)} +\lambda x^{(i)}_k$

$\frac{\partial J}{\partial \theta^{(j)}_k} = \sum ((\theta^{(j)})^Tx^{(i)} - y^{(i,j)})x_k^{(i)} +\lambda \theta^{(j)}_k$

In [6]:
def  cofiCostFunc(params, Y, R, num_users, num_movies, num_features, Lambda):
    """
    Returns the cost and gradient for the collaborative filtering problem
    
    Lambda - regularization parameter
    
    """
        
    # Unfold the params
    X = params[:num_movies*num_features].reshape(num_movies,num_features)
    Theta = params[num_movies*num_features:].reshape(num_users,num_features)
    
    predictions =  X @ Theta.T
    err = (predictions - Y)
    J = 1/2 * np.sum((err**2) * R)
    
    #compute regularized cost function
    reg_X =  Lambda/2 * np.sum(Theta**2)
    reg_Theta = Lambda/2 *np.sum(X**2)
    reg_J = J + reg_X + reg_Theta
    
    # Compute gradient
    X_grad = err*R @ Theta
    Theta_grad = (err*R).T @ X
    grad = np.append(X_grad.flatten(),Theta_grad.flatten())
    
    # Compute regularized gradient
    reg_X_grad = X_grad + Lambda*X
    reg_Theta_grad = Theta_grad + Lambda*Theta
    reg_grad = np.append(reg_X_grad.flatten(),reg_Theta_grad.flatten())
    
    return J, grad, reg_J, reg_grad

In function *cofiCostFunc*: 
 
$X$ (num_movies × n) is the matrix of movie features (also called movie parameters), $n$ is the number of features,  $i$ row of $X$ corresponds to the feature vector $x^{(i)}$ of the $i$ movie.

$Theta$ (num_users × n) is the matrix of user parameters, $j$ row of $Theta$ corresponds to the parameter vector $θ^{(j)}$, of the $j$ user. 

Both $x^{(i)}$ and $θ^{(j)}$ are n-dimensional vectors, $x^{(i)} ∈ R^{n}$ and $θ^{(j)}∈R^{n}$ .

$X$ and $Theta$ will be inicialized randomly latter on, however, in order to test the *cofiCostFunc* function, we have saved some parameters $X$ and $Theta$ in the matlab file *ex8_movieParams.mat*. 


In [7]:
#Load X and Theta from ex8_movieParams.mat and print their dimensions. 

mat2 = loadmat("ex8_movieParams.mat")

X = mat2["X"]  #  (num_movies X num_features)

Theta = mat2["Theta"] #  (num_users X num_features) 

#How many features have the movies ?
print(X.shape[1])
print(X.shape)
print(Theta.shape)

10
(1682, 10)
(943, 10)


In [8]:
# In order to speed up the test, data set size is reduced 
num_users, num_movies, num_features = 4,5,3

#Extract from X, Theta, Y, and R only data coresponding to 
#the new number of users, number of movies, number of features 

X_test = X[:num_movies,:num_features]#(5,3)
Theta_test= Theta[:num_users,:num_features]  #(4,3)
Y_test = Y[:num_movies,:num_users] #(5,4)
R_test = R[:num_movies,:num_users] # (5,4)

print(X_test.shape)
print(Theta_test.shape)
print(Y_test.shape)
print(R_test.shape)

# Due to the simultaneous optimization of X and Theta, 
#all elements of X and Theta are flattened and appended into a single vector
params = np.append(X_test.flatten(),Theta_test.flatten())

# Compute the cost function (without regularization).  ANSWER: J=22.22
print("Cost at loaded parameters (lambda = 0):")
cofiCostFunc(params, Y_test, R_test, num_users, num_movies, num_features, 0)

# Compute the cost function (with regularization). ANSWER: J=31.34
print("Cost at loaded parameters (lambda = 1.5):")
cofiCostFunc(params, Y_test, R_test, num_users, num_movies, num_features, 1.5)

(5, 3)
(4, 3)
(5, 4)
(5, 4)
Cost at loaded parameters (lambda = 0):
Cost at loaded parameters (lambda = 1.5):


(22.224603725685675,
 array([ -2.52899165,   7.57570308,  -1.89979026,  -0.56819597,
          3.35265031,  -0.52339845,  -0.83240713,   4.91163297,
         -0.76677878,  -0.38358278,   2.26333698,  -0.35334048,
         -0.80378006,   4.74271842,  -0.74040871, -10.5680202 ,
          4.62776019,  -7.16004443,  -3.05099006,   1.16441367,
         -3.47410789,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ]),
 31.34405624427422,
 array([ -0.95596339,   6.97535514,  -0.10861109,   0.60308088,
          2.77421145,   0.25839822,   0.12985616,   4.0898522 ,
         -0.89247334,   0.29684395,   1.06300933,   0.66738144,
          0.60252677,   4.90185327,  -0.19747928, -10.13985478,
          2.10136256,  -6.76563628,  -2.29347024,   0.48244098,
         -2.99791422,  -0.64787484,  -0.71820673,   1.27006666,
          1.09289758,  -0.40784086,   0.49026541]))

### Data set of MOVIES 

The list of all movies and their index in the dataset are listed in the ﬁle *movie_idx.txt*

In [18]:
# load the movie list
movieList = open("data/movie_ids.txt","r",encoding = "ISO-8859-1").read().split("\n")[:-1]

#How many movies are collected ?
num_movies = len(movieList)

# see the movie list
movieList

['1 Toy Story (1995)',
 '2 GoldenEye (1995)',
 '3 Four Rooms (1995)',
 '4 Get Shorty (1995)',
 '5 Copycat (1995)',
 '6 Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 '7 Twelve Monkeys (1995)',
 '8 Babe (1995)',
 '9 Dead Man Walking (1995)',
 '10 Richard III (1995)',
 '11 Seven (Se7en) (1995)',
 '12 Usual Suspects, The (1995)',
 '13 Mighty Aphrodite (1995)',
 '14 Postino, Il (1994)',
 "15 Mr. Holland's Opus (1995)",
 '16 French Twist (Gazon maudit) (1995)',
 '17 From Dusk Till Dawn (1996)',
 '18 White Balloon, The (1995)',
 "19 Antonia's Line (1995)",
 '20 Angels and Insects (1995)',
 '21 Muppet Treasure Island (1996)',
 '22 Braveheart (1995)',
 '23 Taxi Driver (1976)',
 '24 Rumble in the Bronx (1995)',
 '25 Birdcage, The (1996)',
 '26 Brothers McMullen, The (1995)',
 '27 Bad Boys (1995)',
 '28 Apollo 13 (1995)',
 '29 Batman Forever (1995)',
 '30 Belle de jour (1967)',
 '31 Crimson Tide (1995)',
 '32 Crumb (1994)',
 '33 Desperado (1995)',
 '34 Doom Generation, The (1995)',
 '35

### Give your own preferences  for some of the movies 
We have ﬁlled out some values according to our own preferences, but you can enter your own movie preferences, so that later when the algorithm runs, you can get your own movie recommendations!

In [19]:
# Initialize my ratings
my_ratings = np.zeros((num_movies,1))

# Create own ratings
my_ratings[0] = 4 
my_ratings[97] = 2
my_ratings[6] = 3
my_ratings[11]= 5
my_ratings[53] = 4
my_ratings[63]= 5
my_ratings[65]= 3
my_ratings[68] = 5
my_ratings[82]= 4
my_ratings[225] = 5
my_ratings[354]= 5

print("New user ratings:\n")
for i in range(len(my_ratings)):
    if my_ratings[i]>0:
        print("Rated",int(my_ratings[i]),"for index",movieList[i])

New user ratings:

Rated 4 for index 1 Toy Story (1995)
Rated 3 for index 7 Twelve Monkeys (1995)
Rated 5 for index 12 Usual Suspects, The (1995)
Rated 4 for index 54 Outbreak (1995)
Rated 5 for index 64 Shawshank Redemption, The (1994)
Rated 3 for index 66 While You Were Sleeping (1995)
Rated 5 for index 69 Forrest Gump (1994)
Rated 4 for index 83 Much Ado About Nothing (1993)
Rated 2 for index 98 Silence of the Lambs, The (1991)
Rated 5 for index 226 Die Hard 2 (1990)
Rated 5 for index 355 Sphere (1998)


Add the additional ratings in the original dataset Y and R

In [20]:
Y = np.hstack((my_ratings,Y))
R = np.hstack((my_ratings!=0,R))

#Confirm that the number of users increased by one
Y.shape

(1682, 944)

### Learning movie recommendations

Normalize the ratings Y. 

Generate random initial values for X and Theta. 

Call Gradient Descent to optimize X and Theta. 

In [30]:
def normalizeRatings(Y, R):
    """
    normalized Y so that each movie has a rating of 0 on average, and returns the 
    mean rating in Ymean.
    """
  
    m = Y.shape[0]  # number of movies
    n = Y.shape[1] # number of users
    
    #Inicialize Ymean and Ynorm as column vectors of 0 with m elements 
    Ymean = np.zeros((m,1))
    Ynorm = np.zeros((m,n))
    
    for i in range(m):
        Ymean[i] = np.sum(Y[i,:])/np.count_nonzero(R[i,:])
        Ynorm[i,R[i,:]==1] = Y[i,R[i,:]==1] - Ymean[i]
        
    return Ynorm, Ymean

In [62]:
def gradientDescent(initial_parameters,Y,R,num_users,num_movies,num_features,alpha,num_iters,Lambda):
    """
    alpha - learning rate
    Optimize X and Theta
    """
    # unfold the initial parameters (consult function cofiCostFunc)
    X = initial_parameters[:num_movies*num_features].reshape(num_movies,num_features)
    Theta = initial_parameters[num_movies*num_features:].reshape(num_users,num_features)
    
    J_history =[]
    
    for i in range(num_iters):
        #Append into a single vector params X and Theta (see the code above)
        params = np.append(X.flatten(),Theta.flatten())
        cost, grad = cofiCostFunc(params, Y, R, num_users, num_movies, num_features, Lambda)[2:]
        
        # unfold grad
        X_grad = grad[:num_movies*num_features].reshape(num_movies,num_features)
        Theta_grad = grad[num_movies*num_features:].reshape(num_users,num_features)
        
    #Update the trainable parameters X and Theta with the classical gradient descent method
        X = X - (alpha * X_grad)
        
        Theta = Theta - (alpha * Theta_grad)
        
        J_history.append(cost)
    
    #Append into a single vector paramsFinal the updated X and Theta
    paramsFinal = np.append(X.flatten(),Theta.flatten())
    return paramsFinal , J_history

In [64]:
# Normalize Ratings
Ynorm, Ymean = normalizeRatings(Y, R)

In [67]:
# number of users
num_users = Ynorm.shape[1]

# number of movies
num_movies = Ynorm.shape[0]

num_features = 10

# Generate randomly initial values for the matrices X and Theta 
#(use for example np.random.randn())
X = np.random.randn(num_movies*num_features)
Theta = np.random.randn(num_users*num_features)

#Append into a single vector params X and Theta 
initial_parameters = np.append(X.flatten(),Theta.flatten())

Lambda = 10
num_iters=400
alpha=0.001

# Optimize parameters using Gradient Descent, use the normalized Ynorm
paramsFinal, J_history = gradientDescent(initial_parameters,Y,R,num_users,num_movies,num_features,alpha,num_iters,Lambda)

### Plot the cost Function

In [72]:
#unfold paramsFinal (consult function cofiCostFunc)
X = paramsFinal[:num_movies*num_features].reshape(num_movies,num_features)
Theta = paramsFinal[num_movies*num_features:].reshape(num_users,num_features)

# Predict all ratings of num_users for num_movies
p = X @ Theta.T

# Extract from p only the recomendations for the added user (the first one)
# Reshape because of rank one problem 

my_scores = p[:,0]

my_predictions = my_scores + Ymean


[[7.46997914 6.54982812 6.20964485 ... 4.30137439 4.4760829  4.42958872]
 [6.79749882 5.87734779 5.53716452 ... 3.62889406 3.80360257 3.75710839]
 [6.62472528 5.70457425 5.36439099 ... 3.45612052 3.63082904 3.58433486]
 ...
 [5.59139195 4.67124092 4.33105765 ... 2.42278719 2.5974957  2.55100152]
 [6.59139195 5.67124092 5.33105765 ... 3.42278719 3.5974957  3.55100152]
 [6.59139195 5.67124092 5.33105765 ... 3.42278719 3.5974957  3.55100152]]


In [79]:
df = pd.DataFrame(np.column_stack((my_predictions,np.array(movieList))))

df.sort_values(by=[0],ascending=False,inplace=True)
df.reset_index(drop=True,inplace=True)

print(df[:10][:])
#Extract only the top 10 recommented movies for the added user 
df = pd.DataFrame(np.column_stack((p[:,:10] + Ymean,np.array(movieList))))

df.sort_values(by=[0],ascending=False,inplace=True)
df.reset_index(drop=True,inplace=True)
print(df[:10][:])

                0                  1                   2     \
0  8.591391946311166  7.671240920017404  7.3310576531479725   
1  8.591391946311166  7.671240920017404  7.3310576531479725   
2  8.591391946311166  7.671240920017404  7.3310576531479725   
3  8.591391946311166  7.671240920017404  7.3310576531479725   
4  8.591391946311166  7.671240920017404  7.3310576531479725   
5  8.591391946311166  7.671240920017404  7.3310576531479725   
6  8.591391946311166  7.671240920017404  7.3310576531479725   
7  8.591391946311166  7.671240920017404  7.3310576531479725   
8  8.591391946311166  7.671240920017404  7.3310576531479725   
9  8.591391946311166  7.671240920017404  7.3310576531479725   

                3                 4                  5                  6     \
0  7.850228475599864  7.73183927691681  7.534293412846973  8.103004103201283   
1  7.850228475599864  7.73183927691681  7.534293412846973  8.103004103201283   
2  7.850228475599864  7.73183927691681  7.534293412846973  8.10300